In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import re
import os

In [18]:
def parse_html(file, index):
    answer = {}
    answer['id'] = index
    url_split = file.find('\n')
    url = file[:url_split]
    answer['url'] = url
    
    text = file[url_split + 1:]
    soup = BeautifulSoup(text, "html5lib")
    name = soup.find('title')
    if name:
        answer['title'] = name.string
    else:
        print("empty title:", index)
    body = soup.find('body')
    if not body:
        print("not found body:", index)
        return answer
    text_prefix = []
    cnt = 0
    for paragraph in body.find_all(['p', 'h1', 'h2', 'h3']):
        cnt += 1
        if paragraph and paragraph.string:
            if len(paragraph.string) > 1:
                text_prefix.append(paragraph.string)
        else:
            if not paragraph.descendants:
                print("not found descedants: ", index)
            for child in paragraph.descendants:
                if child and child.string and len(child.string) > 1:
                    text_prefix.append(child.string)
    rus_text = ' '.join(text_prefix)
    rus_text = re.sub(r"\s+", " ", rus_text)
    answer['text'] = rus_text
    return answer

In [19]:
import pandas as pd

dirname = 'content'
meta = []
pbar = tqdm(range(1, 28027))
for file in os.listdir(dirname):
    with open(os.path.join(dirname, file), mode='r', encoding='utf-8') as fin:
        meta.append(parse_html(fin.read(), file))
    pbar.update(1)    
    
page_info = pd.DataFrame(meta)
page_info.to_csv("all_page_info.csv", index=False)


empty title: 10180.dat
not found body: 10180.dat
empty title: 1059.dat
not found body: 1059.dat


AttributeError: 'NoneType' object has no attribute 'next_element'

In [7]:
import pandas as pd

page_info = pd.DataFrame(meta)
page_info.to_csv("page_info.csv", index=False)

In [36]:
page_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 3 columns):
text     50 non-null object
title    50 non-null object
url      50 non-null object
dtypes: object(3)
memory usage: 1.2+ KB
